# Assignment: Convolutional Neural Networks

## 1. Dataset Exploration (EDA)
We will use the **CIFAR-10** dataset. It consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images.

### Why CIFAR-10?
- **Image-based**: 32x32 RGB images.
- **Multi-class**: 10 distinct classes.
- **Complexity**: Standard benchmark where CNNs significantly outperform dense networks, making it ideal for this comparison.
- **Size**: Fits comfortably in memory.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### Dataset Statistics

In [ ]:
print(f"Training images shape: {train_images.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Number of classes: {len(np.unique(train_labels))}")

### Visualization
Let's view 25 random images from the training set to understand the data diversity.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## 2. Baseline Model (Non-Convolutional)
We will implement a simple Multi-Layer Perceptron (MLP) consisting only of Dense (Fully Connected) layers.

**Architecture:**
- Flatten input (32x32x3 -> 3072)
- Dense Layer (512 units, ReLU)
- Dense Layer (256 units, ReLU)
- Output Layer (10 units, Softmax)

In [ ]:
baseline_model = models.Sequential([
    layers.Flatten(input_shape=(32, 32, 3)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])

baseline_model.summary()

### Training the Baseline

In [ ]:
baseline_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

baseline_history = baseline_model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

## 3. Convolutional Architecture Design
Now we design a CNN from scratch. 

**Architecture:**
- **Conv2D**: 32 filters, 3x3 kernel. Extracts low-level features.
- **MaxPooling2D**: 2x2. Reduces spatial dimensions.
- **Conv2D**: 64 filters, 3x3 kernel. Extracts higher-level features.
- **MaxPooling2D**: 2x2.
- **Conv2D**: 64 filters, 3x3 kernel.
- **Flatten**: Converts 3D volume to 1D vector.
- **Dense**: 64 units, ReLU. Interpretation layer.
- **Output**: 10 units. Classification.

**Justification:**
We use a stack of Conv2D and MaxPooling2D layers to progressively extract features and downsample the image. The number of filters increases (32 -> 64) as we go deeper to capture more complex patterns.

In [ ]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add Dense layers on top
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(64, activation='relu'))
cnn_model.add(layers.Dense(10))

cnn_model.summary()

### Training the CNN


In [ ]:
cnn_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

cnn_history = cnn_model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))